<a href="https://colab.research.google.com/github/mstantongeddes/Scrummarize_EC_Consultations/blob/main/Summarise_EC_consultation_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Scrape and Summarise Feedback Provided to European Commission Public Consultations

##Access PDF files and extract Text

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/EC_consultations_scrapeAndsummarise/

/content/drive/MyDrive/EC_consultations_scrapeAndsummarise


In [5]:
!pwd


/content


In [6]:
!pip install PyPDF2

import PyPDF2

with open('/content/drive/MyDrive/consultations_summariser/files/090166e50d7f0604.pdf', 'rb') as pdf_file:
  reader = PyPDF2.PdfReader(pdf_file)
  number_of_pages = len(reader.pages)

   #Initialize an empty string to hold all the text

  parsed_text = ""

  #Loop through all the pages and concatenate the text
  for pagenumber in range(number_of_pages):
    page = reader.pages[pagenumber]
    parsed_text += page.extract_text() + "\n"



In [8]:
print(parsed_text)

 
1 
 
SECTION 1  – Setting the scene: The European aviation market and 
airports  
 
1. Market developments post COVID  
 
The European aviation market is characterized by  a strong but uneven recovery post CO VID. 
Last year, passenger numbers increased just under 20% compared to 2022  coming  close to 
a full recovery compared to 2019 numbers (-5%). 
  
While there is overall  a good recovery dynamic o f passenger traffic  at network level , we are 
seeing significant performance gaps across national  market s and individual airports . This 
reflects the impact of geopolitics and the war in Ukraine  for countries such as Finland or the 
Czech Republic as well as new patterns of air connectivity and structural changes in the 
aviation market  - as traffic recovery is driven by leisure and V isiting Friends and Relatives  
(VFR) demand  as well as  the growth of ultra-LCCs which account for  most of the growth in 
many Southern European countries .   
 
Across the different segments o

##Prepare LLM model

In [5]:
!pip install -r requirements.txt

In [9]:
import json
import torch
from  transformers import (AutoTokenizer,
                           AutoModelForCausalLM,
                           BitsAndBytesConfig,
                           pipeline)



Proivide HuggingFace access credentials

In [12]:
config_data = json.load(open("config.json"))

HF_TOKEN = config_data["HF_TOKEN"]

Quantisation configuration

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
  )

Shorten text to not exceed max length for summarizer

In [10]:
#Shorten the input text before processing
max_input_length = 1000  # Adjust this value as needed
truncated_parsed_text = parsed_text[:max_input_length]

print(truncated_parsed_text)

 
1 
 
SECTION 1  – Setting the scene: The European aviation market and 
airports  
 
1. Market developments post COVID  
 
The European aviation market is characterized by  a strong but uneven recovery post CO VID. 
Last year, passenger numbers increased just under 20% compared to 2022  coming  close to 
a full recovery compared to 2019 numbers (-5%). 
  
While there is overall  a good recovery dynamic o f passenger traffic  at network level , we are 
seeing significant performance gaps across national  market s and individual airports . This 
reflects the impact of geopolitics and the war in Ukraine  for countries such as Finland or the 
Czech Republic as well as new patterns of air connectivity and structural changes in the 
aviation market  - as traffic recovery is driven by leisure and V isiting Friends and Relatives  
(VFR) demand  as well as  the growth of ultra-LCCs which account for  most of the growth in 
many Southern European countries .   
 
Across the different segments o

##Load the Tokenizer and the LLM

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "facebook/bart-large-cnn"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=HF_TOKEN,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
     # Use a custom device_map to offload to CPU
    device_map={'': 0}  # Assuming you have a single GPU with ID 0
    #device_map="auto", #Original code
)

messages = [
    {"role": "system", "content": "You are an assistant that shortens text."},
    {"role": "user", "content": f"Please shorten this text: {truncated_parsed_text}"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Sure! Here is a shorter version of the text:

The European aviation market is recovering, but unevenly. Passenger numbers increased 19% in 2023 compared to 2022, close to a full recovery. However, there are significant performance gaps across countries and airports due to geopolitical factors and structural changes in the market. Leisure and VFR demand, as well as the growth of ultra-low-cost carriers, are driving traffic recovery in many Southern European countries.


*Assistant Chat to give instructions*

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", "HuggingFaceH4/zephyr-7b-beta")
messages = [
    {
        "role": "system",
        "content": "You are an assistant that shortens text.",
    },
    {"role": "user", "content": f"Please shorten this text: {truncated_parsed_text}"},
]
print(pipe(messages, max_new_tokens=128)[0]['generated_text'][-1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]